In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [2]:
x_b_train = pd.read_csv('data_b_train.csv')
x_b_test = pd.read_csv('data_b_test.csv')

y_train = pd.read_csv('y_train.csv')

In [3]:
# b表使用特征
b_col = ['id', 'x_num_0', 'x_num_1', 'x_num_3', 'x_num_4',
         'x_cat_0', 'x_cat_2', 'x_cat_4', 'x_cat_8', 'x_cat_11', 'x_cat_12']

x_b_train = x_b_train[b_col]
x_b_test = x_b_test[b_col]

origin_test_id = x_b_test['id']

In [4]:
# 合并b表和target
train = pd.merge(x_b_train,y_train,how='inner',on='id')

# train = train[(train['x_num_0']!=-99)&(train['x_num_1']!=-99)&(train['x_num_3']!=-99)]

In [5]:
x_m_train = pd.read_csv('data_m_train.csv')
x_m_test = pd.read_csv('data_m_test.csv')

In [6]:
# test中m表缺失值处理

x_m_test.replace(-99.0, np.nan, inplace=True)

In [7]:
m_col = ['id', 'x_num_5','x_num_6', 'x_num_9', 'x_num_11', 'x_num_13', 'x_num_16',
         'x_num_18', 'x_num_20', 'x_num_22', 'x_num_25','x_num_26','x_num_30', 'x_num_31', 'x_num_32', 'x_num_34', 'x_num_35', 
         'x_num_37', 'x_num_39', 'x_num_41', 'x_num_42', 'x_num_43', 'x_num_45', 'x_num_46', 'x_num_50', 'x_num_51', 'x_num_52', 
         'x_num_54', 'x_num_58', 'x_num_60','x_num_61', 'x_num_62', 'x_num_65', 'x_num_68', 'x_num_69','x_num_70']

In [8]:
# 选取相同id下最大的timestamp,最大值，标准差，离散系数

def changeName(df, name):
    new = []
    for col in df.columns.values:
        if col is 'id':
            new.append(col)
        else:
            new.append(col+'_'+name)
    return new
    


x_m_train_mean = x_m_train[m_col].groupby('id', as_index=False).mean()
x_m_train_time = x_m_train[m_col+['timestamp']].sort_values('timestamp', ascending=False).groupby('id', as_index=False).first()
x_m_train_time.drop('timestamp',axis=1, inplace=True)
x_m_train_max = x_m_train[m_col].groupby('id', as_index=False).max()
x_m_train_std = x_m_train[m_col].groupby('id', as_index=False).std()
x_m_train_cov = x_m_train_std / x_m_train_mean


x_m_train_time.columns = changeName(x_m_train_time, 'time')
x_m_train_max.columns = changeName(x_m_train_max, 'max')
x_m_train_std.columns = changeName(x_m_train_std, 'std')
x_m_train_std['id']=x_m_train_max['id']
x_m_train_cov.columns = changeName(x_m_train_cov, 'cov')
x_m_train_cov['id']=x_m_train_max['id']


    
x_m_train = pd.merge(x_m_train_time, x_m_train_max, how='left',on='id')
x_m_train = pd.merge(x_m_train, x_m_train_std, how='left',on='id')
x_m_train = pd.merge(x_m_train, x_m_train_cov, how='left',on='id')



x_m_test_mean = x_m_test[m_col].groupby('id', as_index=False).mean()
x_m_test_time = x_m_test[m_col+['timestamp']].sort_values('timestamp', ascending=False).groupby('id', as_index=False).first()
x_m_test_time.drop('timestamp',axis=1, inplace=True)
x_m_test_max = x_m_test[m_col].groupby('id', as_index=False).max()
x_m_test_std = x_m_test[m_col].groupby('id', as_index=False).std()
x_m_test_cov = x_m_test_std / x_m_test_mean


x_m_test_time.columns = changeName(x_m_test_time, 'time')
x_m_test_max.columns = changeName(x_m_test_max, 'max')
x_m_test_std.columns = changeName(x_m_test_std, 'std')
x_m_test_std['id']=x_m_test_max['id']
x_m_test_cov.columns = changeName(x_m_test_cov, 'cov')
x_m_test_cov['id']=x_m_test_max['id']

    
x_m_test = pd.merge(x_m_test_time, x_m_test_max, how='left',on='id')  
x_m_test = pd.merge(x_m_test, x_m_test_std, how='left',on='id')
x_m_test = pd.merge(x_m_test, x_m_test_cov, how='left',on='id')

In [9]:
# 将b表与m表拼接
train = pd.merge(train, x_m_train, how='left',on='id')
test = pd.merge(x_b_test, x_m_test, how='left',on='id')

In [10]:
# test中b表缺失值处理

test.replace(-99.0, 0, inplace=True)

In [11]:
y_train = train['target']
train.drop("target", axis=1, inplace=True)
test_id = test['id']

x_train = train.drop('id', axis=1)
x_test = test.drop('id', axis=1)

In [12]:
params = {'boosting':'gbdt',
          'objective':'binary',
          'max_depth':6,
          'learning_rate':0.01,
          'num_leaves':36,
          'metric':'auc',
         'min_child_samples':9,
          'min_child_weight':0.001,
          'bagging_fraction':0.6, 
          'feature_fraction':0.5,
          'reg_alpha':0.5,
          'reg_lambda':0.01,
          'cat_smooth': 0
         }



data_train = lgb.Dataset(x_train, y_train, silent=True)
cv_results = lgb.cv(
    params, lgb.Dataset(x_train, y_train, silent=True), 
    num_boost_round=5000, nfold=5, stratified=False, shuffle=True, metrics='auc',
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=666)

print('best n_estimators:', len(cv_results['auc-mean']))
print('best cv score:', cv_results['auc-mean'][-1])

# 0.858

[LightGBM] [Info] Number of positive: 1304, number of negative: 10924
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19467
[LightGBM] [Info] Number of data points in the train set: 12228, number of used features: 134
[LightGBM] [Info] Number of positive: 1274, number of negative: 10954
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19467
[LightGBM] [Info] Number of data points in the train set: 12228, number of used features: 134
[LightGBM] [Info] Number of positive: 1294, number of negative: 10934
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004274 seconds.
You can set `force_row_wise=tru

In [13]:
# 构建模型
train_data = lgb.Dataset(x_train, label=y_train)

# params = {'boosting':'gbdt',
#           'objective':'binary',
#           'max_depth':6,
#           'learning_rate':0.01,
#           'num_leaves':50,
#           'metric':['auc', 'binary_logloss'],
#           'verbose':1,
#          }

# my_model = lgb.train(params, train_data, 500)


params = {'boosting':'gbdt',
          'objective':'binary',
          'max_depth':6,
          'learning_rate':0.01,
          'num_leaves':36,
          'metric':'auc',
         'min_child_samples':9,
          'min_child_weight':0.001,
          'bagging_fraction':0.6, 
          'feature_fraction':0.5,
          'reg_alpha':0.5,
          'reg_lambda':0.01,
          'cat_smooth': 0
         }
    
my_model = lgb.train(params, train_data, 870)

predictions = my_model.predict(x_test)

[LightGBM] [Info] Number of positive: 1602, number of negative: 13684
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19467
[LightGBM] [Info] Number of data points in the train set: 15286, number of used features: 134
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.104802 -> initscore=-2.144974
[LightGBM] [Info] Start training from score -2.144974
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [14]:
# 产生结果
submission = pd.DataFrame(
    predictions,
    index=origin_test_id,
    columns=['target'])
submission.to_excel(
    'LGBM_param_std.xlsx', index_label='id', sheet_name='sheetname')

In [15]:
len(predictions)

2794

In [16]:
from sklearn.metrics import roc_auc_score

# 查看在抽取训练集上的准确率


y_train_true = y_train

y_train_predict = my_model.predict(x_train)
score = roc_auc_score(y_train_true, y_train_predict)
print(score)
print(len(y_train_predict))
# 0.9462020763643665
# 14356
y_train_predict[0]

0.9799049967137687
15286


0.4593565178707095